## 긍정, 부정의 감성분석 : 데이터 양을 늘리기

In [21]:
import pandas as pd

# 두 파일을 병합하기
df_train = pd.read_csv('ratings_test.csv')
df_test = pd.read_csv('ratings_train.csv')

df = pd.concat([df_train, df_test])

In [22]:
# 중복 제거
df.drop_duplicates(subset=['text'], inplace=True)
# 결측치 제거
df = df.dropna(how='any')

In [23]:
# 중복과 결측치를 제거한 데이터를 파일로 저장하기
df.to_csv('review_filtered.csv')

In [25]:
df = pd.read_csv("review_filtered.csv")
# 데이터를 라벨 0,1의 비중을 맞춰서 1000를 추출

df = df[['text','label']]

df_pos = df[df['label'] == 1]
df_neg = df[df['label'] == 0]

sample_pos = df_pos.sample(n=300, random_state=12)
sample_neg = df_neg.sample(n=300, random_state=12)

sample_df = pd.concat([sample_pos,sample_neg], ignore_index=True)

print("샘플링된 데이터 개수 : ", len(sample_df))
print("라벨별 데이터 개수 : \n", sample_df['label'].value_counts())

df = sample_df

샘플링된 데이터 개수 :  600
라벨별 데이터 개수 : 
 label
1    300
0    300
Name: count, dtype: int64


In [26]:
# 한글 형태소 분석기를 통해, 영화 리뷰 문장들을 전처리
# 한글만 남기고 모두 제거하기
df['text'] = df['text'].str.replace("[^가-힣 ]","",regex=True)
# '빈칸'만 남은 데이터는 공백으로 변경
df['text'] = df['text'].str.replace("^ +","",regex=True)
# 공백만 있는 데이터는 Null / Nan으로 변경
import numpy as np
df['text'] = df['text'].replace("", np.nan)

In [27]:
# df에서 null 제거
df = df.dropna(how='any')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    600 non-null    object
 1   label   600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB
None


In [28]:
stopwords = [
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', 
    '고', '면', '며', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', 
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '일', '문제', '경우', '부분',
    '내용', '결과', '자체', '가지','에요',
    '뿐', '대로', '만큼', '만', '지', '따름', '나름', '김에', '터', '너무', '어요'
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니'
]

In [29]:
from konlpy.tag import Okt
okt = Okt()

# 훈련 데이터 만들기
train_data = []
for i in range(len(df)):
    emotion = df.iloc[i]['label']
    text = df.iloc[i]['text']
    # 문장을 형태소 분석하기 -> 문장을 단어(토큰)의 리스트로 변환 
    tokenized_text = okt.morphs(text)
    # 형태소 분석결과에서 불용어를 제거함
    filtered_tokens = [ word for word in tokenized_text if not word in stopwords ]
    # (토큰화 된 문장, 감정) 형태의 튜플을 리스트에 저장
    train_data.append((filtered_tokens,emotion))

In [30]:
all_tokens = []
for tokens, emotion in train_data :
    for token in tokens :
        all_tokens.append(token)
vocabulary = list(set(all_tokens))

In [31]:
def create_feature_set(tokens, vocabulary) :
    token_set = set(tokens)
    features = { word : (word in token_set) for word in vocabulary }
    return features

In [32]:
train_feature_datas = []
train_feature_datas = [
    (create_feature_set(tokens,vocabulary),emotion)
    for tokens, emotion in train_data ]

In [33]:
import nltk
model = nltk.NaiveBayesClassifier.train(train_feature_datas)

In [34]:
def classify_text(text, model, tokenizer, vocabulary) :
    tokens = tokenizer.morphs(text)
    filtered_tokens = [ word for word in tokens if not word in stopwords ]
    features = create_feature_set(filtered_tokens, vocabulary)
    clasified_label = model.classify(features)
    print(model.prob_classify(features).prob(clasified_label))
    result = "부정" if clasified_label == -1 else "긍정"
    print("예측 결과 : ", result)
    return result

In [35]:
text_list = [
    "깨긋하구 아늑한분위기",
    "조용하고 주위에 볼거리도 많아요",
    "방이 작고 방음 안되요 주차장 찾기 어려워요",
    "3명이상 숙박하기에는 좋지 않습니다",
    "사장님도 친절하시고 숙소 안도 따뜻해서 좋았어요",
    "친절하고 깔끔해서 지내기 좋았습니다",
    "연박으로 묵었는데 크리스마스 너무 잘 보냈습니다 사장님 친절하시고 시설도 괜찮습니다",
    "친절하시고 위치도 경기전 근처라 한옥마을 다니기도 편했습니다",
    "딱 1박용으로 괜찮은 곳이에요",
    "특별한 서비스는 없었어요"
]
df = pd.read_csv('1_1400_output_partial.csv')

for text in df['sentence'] :
    classify_text(text, model, okt, vocabulary)

0.9968752908973735
예측 결과 :  긍정
0.9999976600516886
예측 결과 :  긍정
0.9999999999795115
예측 결과 :  긍정
0.9999999224424134
예측 결과 :  긍정
0.9999968392840076
예측 결과 :  긍정
0.9999998522213104
예측 결과 :  긍정
0.9999998522213104
예측 결과 :  긍정
0.9999999994454329
예측 결과 :  긍정
0.9999999997063849
예측 결과 :  긍정
0.9999999980397286
예측 결과 :  긍정
0.9996784064732025
예측 결과 :  긍정
0.9999999989057594
예측 결과 :  긍정
0.9999998358587681
예측 결과 :  긍정
0.9999997457346884
예측 결과 :  긍정
0.9999653150329436
예측 결과 :  긍정
0.9991970187391248
예측 결과 :  긍정
1.0
예측 결과 :  긍정
0.9999999999949173
예측 결과 :  긍정
0.9999999999999409
예측 결과 :  긍정
0.8860097116236841
예측 결과 :  긍정
0.9985674620212551
예측 결과 :  긍정
0.9994924615880645
예측 결과 :  긍정
0.620558173852381
예측 결과 :  긍정
0.9999999769805626
예측 결과 :  긍정
0.9999976233542517
예측 결과 :  긍정
0.9996867309408127
예측 결과 :  긍정
0.9998474195448249
예측 결과 :  긍정
0.8109491702035511
예측 결과 :  긍정
0.9888237397223412
예측 결과 :  긍정
0.9901033779219527
예측 결과 :  긍정
0.999806376089473
예측 결과 :  긍정
0.999915798107608
예측 결과 :  긍정
0.9995530141462857
예측 결과 :

KeyboardInterrupt: 